In [2]:
import os
import re
import shutil
import subprocess
from pathlib import Path

def update_version(version_bump='patch'):
    """
    Update version numbers across the project.
    version_bump can be 'major', 'minor', or 'patch'
    """
    # Read current version from pyproject.toml
    with open('pyproject.toml', 'r') as f:
        content = f.read()
        version_match = re.search(r'version = "(\d+)\.(\d+)\.(\d+)"', content)
        if not version_match:
            raise ValueError("Version not found in pyproject.toml")
        
        major, minor, patch = map(int, version_match.groups())
        print(f"Current version: {major}.{minor}.{patch}")
        
    # Increment version based on bump type
    if version_bump == 'major':
        major += 1
        minor = 0
        patch = 0
    elif version_bump == 'minor':
        minor += 1
        patch = 0
    else:  # patch
        patch += 1
        
    new_version = f"{major}.{minor}.{patch}"
    print(f"New version: {new_version}")
    
    # Update pyproject.toml
    new_content = re.sub(
        r'version = "\d+\.\d+\.\d+"',
        f'version = "{new_version}"',
        content
    )
    with open('pyproject.toml', 'w') as f:
        f.write(new_content)
        
    # Update __init__.py
    init_path = Path('src/opengradient/__init__.py')
    with open(init_path, 'r') as f:
        content = f.read()
    
    new_content = re.sub(
        r'__version__ = "\d+\.\d+\.\d+"',
        f'__version__ = "{new_version}"',
        content
    )
    with open(init_path, 'w') as f:
        f.write(new_content)
        
    # Delete dist directory if it exists
    dist_path = Path('dist')
    if dist_path.exists():
        shutil.rmtree(dist_path)
        print("Deleted dist directory")
        
    # Run build
    result = subprocess.run(['python', '-m', 'build'], capture_output=True, text=True)
    print("Build output:", result.stdout)
    if result.stderr:
        print("Build errors:", result.stderr)
        
    # Update notebook install command
    notebook_path = Path('infer.ipynb')
    if notebook_path.exists():
        with open(notebook_path, 'r') as f:
            content = f.read()
            
        # Update pip install command in notebook
        new_content = re.sub(
            r'!pip install opengradient==\d+\.\d+\.\d+',
            f'!pip install opengradient=={new_version}',
            content
        )
        with open(notebook_path, 'w') as f:
            f.write(new_content)
            
    print(f"\nVersion updated to {new_version}")
    return new_version

# Example usage in notebook:
update_version('patch')  # For patch version bump
# update_version('minor')  # For minor version bump
# update_version('major')  # For major version bump

Current version: 0.3.29
New version: 0.3.30
Deleted dist directory
Build output: * Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools>=61.0
* Getting build dependencies for sdist...
running egg_info
writing src/opengradient.egg-info/PKG-INFO
writing dependency_links to src/opengradient.egg-info/dependency_links.txt
writing entry points to src/opengradient.egg-info/entry_points.txt
writing requirements to src/opengradient.egg-info/requires.txt
writing top-level names to src/opengradient.egg-info/top_level.txt
reading manifest file 'src/opengradient.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'src/opengradient.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src/opengradient.egg-info/PKG-INFO
writing dependency_links to src/opengradient.egg-info/dependency_links.txt
writing entry points to src/opengradient.egg-info/entry_points.txt
wr

'0.3.30'

In [3]:
!pip uninstall -y opengradient
!pip install -e . 

Found existing installation: opengradient 0.3.29
Uninstalling opengradient-0.3.29:
  Successfully uninstalled opengradient-0.3.29
Obtaining file:///Users/oliver/Desktop/code/opengradient/sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for opengradient (pyproject.toml) ... done
  Created wheel for opengradient: filename=opengradient-0.3.30-0.editable-py3-none-any.whl size=5587 sha256=0a044d8bf268227a9dd388566e813d2d6f2403a244b9f8847b0c2e3180499f0a
  Stored in directory: /private/var/folders/n0/8l2p0mjx6b74dh_nd5rp9vs00000gn/T/pip-ephem-wheel-cache-ol_5pmex/wheels/19/aa/e2/93e5f9c5d585eea6876f315008bd8814e9550592b29567ec34
Successfully built opengradient


In [1]:
import opengradient as og
from opengradient.types import HistoricalInputQuery, CandleType, CandleOrder  # Changed import path

og.init(
    private_key = "1404c83e29d18180d33dac46848939a38ed9fc3c0c869f7623f0eb3a05504055",
    email = "oliver@opengradient.ai",
    password = r'9^D.fMNwDGL7f5\Xk$+r'
)

input_query = HistoricalInputQuery(
    base="ETH",
    quote="USD",
    total_candles=10,
    candle_duration_in_mins=30,
    order=CandleOrder.ASCENDING,
    candle_types=[
        CandleType.OPEN,
        CandleType.HIGH,
        CandleType.LOW,
        CandleType.CLOSE
    ]
)


scheduler_params = {
    'frequency': 3600,  # Every hour
    'duration_hours': 72  # 3 days
}

print("Deploying workflow with updated query structure...")
model_cid = "QmP4BeRjycVxfKBkFtwj5xAa7sCWyffMQznNsZnXgYHpFX"
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="ohlc_3h_candles",
    scheduler_params=scheduler_params
)

print(f"Contract deployed at: {contract_address}")

# Run and check results
tx_hash = og.run_workflow(contract_address)
print(f"Run transaction hash: {tx_hash}")

# Wait for execution
import time
time.sleep(30)

result = og.read_workflow_result(contract_address)
print("Workflow result:", result)

/opt/miniconda3/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.2 at infer.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


Deploying workflow with updated query structure...
📦 Deploying workflow contract...


Web3TypeError: One or more arguments could not be encoded to the necessary ABI type. Expected types are: string, (string,uint32,uint32,uint8,uint8[]), address, string

In [3]:
og.create_model("test123456", "test", "requirements.txt")

{'name': 'test123456',
 'versionString': '0.01',
 'upload': {'model_cid': 'Qmf88ozrWNyxvtjg3Djko8bZRDuCsAbFu4LuFNjKHbb6Lr',
  'size': 2800}}

In [3]:
from opengradient.types import HistoricalInputQuery, CandleOrder, CandleType, SchedulerParams

# Create input query using proper types
input_query = HistoricalInputQuery(
    currency_pair="ETH/USD",
    total_candles=10,
    candle_duration_in_mins=30,
    order=CandleOrder.ASCENDING,
    candle_types=[
        CandleType.OPEN,
        CandleType.HIGH,
        CandleType.LOW,
        CandleType.CLOSE
    ]
)

# Create scheduler parameters directly using SchedulerParams class
scheduler_params = SchedulerParams(
    frequency=3600,  # Run every hour (3600 seconds)
    duration_hours=720  # Run for approximately 1 month (30 days * 24 hours)
)

# Deploy a new workflow with your model
model_cid = "QmRhcpDXfYCKsimTmJYrAVM4Bbvck59Zb2onj3MHv9Kw5N"
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="open_high_low_close",
    scheduler_params=scheduler_params
)

og.run_workflow(contract_address)

#Read the workflow result

result = og.read_workflow_result(contract_address)
print("Inference result:", result)

📦 Deploying workflow contract...
✅ Workflow contract deployed at: 0x00893E1E45a178F58f0683F8FF902F26F488060a

⏰ Setting up automated execution schedule...
   • Frequency: Every 3600 seconds
   • Duration: 720 hours
   • End Time: 2025-03-06 10:18:43
❌ Failed to set up automated execution schedule
   Error: {'code': -32000, 'message': 'tx failed with code: 1: raw_log: failed to simulate transaction: revert: 0x: Reverted: EVMCall failed: EVMCall failed [cosmossdk.io/errors@v1.0.1/errors.go:151]: invalid request'}
   The workflow contract is still deployed and can be executed manually.


KeyboardInterrupt: 

In [4]:
from opengradient.types import HistoricalInputQuery, CandleOrder, CandleType, SchedulerParams

# Deploy workflow
input_query = {
    'currency_pair': "SUI/USDT",
    'total_candles': 6,     
    'candle_duration_in_mins': 180,  
    'order': "Ascending",   
    'candle_types': ["Open", "High", "Low", "Close"]
}

# Create scheduler parameters directly using SchedulerParams class
scheduler_params = SchedulerParams(
    frequency=3600,  # Run every hour (3600 seconds)
    duration_hours=720  # Run for approximately 1 month (30 days * 24 hours)
)

print("Deploying workflow...")
model_cid = "QmP4BeRjycVxfKBkFtwj5xAa7sCWyffMQznNsZnXgYHpFX"
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="ohlc_3h_candles",
    scheduler_params=scheduler_params
)
print(f"Contract deployed at: {contract_address}")

Deploying workflow...
📦 Deploying workflow contract...
✅ Workflow contract deployed at: 0xaF7BaD4bA1F831ef037F597f66A8a58ecf4A04B4

⏰ Setting up automated execution schedule...
   • Frequency: Every 3600 seconds
   • Duration: 720 hours
   • End Time: 2025-02-13 16:29:12
✅ Automated execution schedule set successfully!
   Transaction hash: a6eb1b98841d0a22fd89737a5adaf06170d84a77fb13d872fc2ec5aaf7f941cb
Contract deployed at: 0xaF7BaD4bA1F831ef037F597f66A8a58ecf4A04B4


In [6]:

# Test inference
test_input = {"X": [2]}  # Input value of 2.0 should give output of 4.0
result = og.infer(
    model_cid="QmPZkp4ZGMPKkQSw3f8h21qpeFd3YeB3MfvXxHqmTrg2C1",
    model_input=test_input,
    inference_mode=og.InferenceMode.VANILLA
)
print(f"ZK Inference result: {result}")

ERROR:web3.manager.RequestManager:An RPC error was returned by the node. Check the message provided in the error and any available logs for more information.


ZK Inference result: ('f22972aab598427db5a71726a677210dd467af613dfae1bb5ca0ab97168183fd', {'Y': array([4.], dtype=float32)})


In [11]:
# Define input query for SUI/USD historical data
input_query = {
    'currency_pair': "SUI/USD",
    'total_candles': 10,  # Model requires 10 candles
    'candle_duration_in_mins': 30,  # 30-minute candles
    'order': "Ascending",  # Time-ordered, oldest to newest
    'candle_types': ["Open", "High", "Low", "Close"]  # OHLC in this order
}

# Deploy workflow with the SUI/USD model
model_cid = "QmY1RjD3s4XPbSeKi5TqMwbxegumenZ49t2q7TrK7Xdga4"  # SUI/USD model CID
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="open_high_low_close"  # Same input tensor name as specified
)
print(f"SUI/USD forecast workflow deployed at: {contract_address}")

tx_hash = og.run_workflow(contract_address)
print(f"Run transaction completed {tx_hash}")

# Read the workflow result (forecast return)
result = og.read_workflow_result(contract_address)
print("Forecast 30-minute return:", result)

SUI/USD forecast workflow deployed at: 0x66efd8e568C5a5EaD149be03307fa24C649c82b2
Run transaction completed ([('destandardized_prediction', [(-2060880360659211874008178711, 31)], [1])], [], [], False)
Forecast 30-minute return: ([('destandardized_prediction', [(-2060880360659211874008178711, 31)], [1])], [], [], False)


In [3]:
from opengradient.types import HistoricalInputQuery, CandleOrder, CandleType, SchedulerParams

# Define historical price data requirements
input_query = HistoricalInputQuery(
    currency_pair="ETH/USD",
    total_candles=10,
    candle_duration_in_mins=30,
    order=CandleOrder.ASCENDING,
    candle_types=[
        CandleType.OPEN,
        CandleType.HIGH,
        CandleType.LOW,
        CandleType.CLOSE
    ]
)

# Set up automated execution schedule
scheduler_params = SchedulerParams(
    frequency=1800,  # Run every 30 minutes (1800 seconds)
    duration_hours=720  # Run for 1 month (30 days * 24 hours)
)

# Deploy a new workflow with your model
model_cid = "QmY1RjD3s4XPbSeKi5TqMwbxegumenZ49t2q7TrK7Xdga4"
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="open_high_low_close",
    scheduler_params=scheduler_params
)

📦 Deploying workflow contract...
✅ Workflow contract deployed at: 0x6c95E170A0821c390f6B7799E9b3C4c803297e7C

⏰ Setting up automated execution schedule...
   • Frequency: Every 1800 seconds
   • Duration: 720 hours
   • End Time: 2025-02-13 14:00:20
✅ Automated execution schedule set successfully!
   Transaction hash: b187244358db088b5a7f675d9c777dd428d3009caf837b0dbc1ccc05d703610a


In [25]:
import opengradient as og
import time
from web3.exceptions import TimeExhausted, Web3RPCError

def verify_oracle_data(contract_address):
    """Check oracle contract and data"""
    try:
        # Get the contract instance
        contract = og._client._w3.eth.contract(
            address=contract_address,
            abi=og._client._get_model_executor_abi()
        )
        
        # Get the historical contract address
        oracle_address = contract.functions.historicalContract().call()
        print(f"\nOracle Contract Address: {oracle_address}")
        
        # Get inference result to check if data is ready
        result = contract.functions.getInferenceResult().call()
        if result:
            numbers, strings, jsons, is_simulation = result
            print("\nCurrent Inference Result:")
            print(f"Numbers tensors: {len(numbers)}")
            print(f"String tensors: {len(strings)}")
            print(f"JSON scalars: {len(jsons)}")
            print(f"Is simulation: {is_simulation}")
            
        return oracle_address
        
    except Exception as e:
        print(f"Error checking oracle: {e}")
        return None

# Deploy workflow
input_query = {
    'currency_pair': "SUI/USDT",
    'total_candles': 6,     
    'candle_duration_in_mins': 180,  
    'order': "Ascending",   
    'candle_types': ["Open", "High", "Low", "Close"]
}

print("Deploying workflow...")
model_cid = "QmP4BeRjycVxfKBkFtwj5xAa7sCWyffMQznNsZnXgYHpFX"
contract_address = og.new_workflow(
    model_cid=model_cid,
    input_query=input_query,
    input_tensor_name="ohlc_3h_candles"
)
print(f"Contract deployed at: {contract_address}")

# Wait a bit for oracle
print("Waiting for oracle data...")
time.sleep(30)

# Verify the oracle data
oracle_data = verify_oracle_data(contract_address)

# Compare with expected format
expected_shape = [[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]
print("\nExpected Data Shape:")
print(f"Number of rows: {len(expected_shape)}")
print(f"Number of columns: {len(expected_shape[0])}")

if oracle_data:
    print("\nActual Data Shape:")
    if isinstance(oracle_data, (list, tuple)):
        print(f"Number of rows: {len(oracle_data)}")
        if len(oracle_data) > 0 and isinstance(oracle_data[0], (list, tuple)):
            print(f"Number of columns: {len(oracle_data[0])}")

Deploying workflow...
Contract deployed at: 0x3F8FBC82246e06Ef684cbfA0a08c243956F07448
Waiting for oracle data...

Oracle Contract Address: 0x00000000000000000000000000000000000000F5

Current Inference Result:
Numbers tensors: 0
String tensors: 0
JSON scalars: 0
Is simulation: False

Expected Data Shape:
Number of rows: 6
Number of columns: 4

Actual Data Shape:


In [23]:
# # Define input query for SUI/USD historical data
# input_query = {
#     'currency_pair': "SUI/USD",
#     'total_candles': 6,     # Model requires 6 candles
#     'candle_duration_in_mins': 180,  # 3-hour candles (180 minutes)
#     'order': "Ascending",   # Time-ordered, oldest to newest
#     'candle_types': ["Open", "High", "Low", "Close"]  # OHLC in this order
# }

# # Deploy workflow with the SUI/USD 6h model
# model_cid = "QmP4BeRjycVxfKBkFtwj5xAa7sCWyffMQznNsZnXgYHpFX"  # 6h forecast model CID
# contract_address = og.new_workflow(
#     model_cid=model_cid,
#     input_query=input_query,
#     input_tensor_name="ohlc_3h_candles"  # Note the different tensor name from previous model
# )
# print(f"SUI/USD 6h forecast workflow deployed at: {contract_address}")

# Run the workflow
result = og.run_workflow(contract_address)
print(f"Run transaction completed")

# Read the workflow result (forecast return)
result = og.read_workflow_result(contract_address)
print("Forecast 6-hour return:", result)

ERROR:web3.manager.RequestManager:An RPC error was returned by the node. Check the message provided in the error and any available logs for more information.


Web3RPCError: {'code': -32000, 'message': 'tx failed with code: 1: raw_log: failed to simulate transaction: revert: 0x: Reverted: EVMCall failed: EVMCall failed [cosmossdk.io/errors@v1.0.1/errors.go:151]: invalid request'}